# 02. 공행성 탐지 (Comovement Detection)

이 노트북은 100개 품목 간의 선행-후행 관계를 탐지합니다.

## 분석 방법
1. **CCF (Cross-Correlation Function)** - 시차별 상관관계
2. **Granger Causality Test** - 방향성 있는 인과관계
3. **DTW (Dynamic Time Warping)** - 비선형 패턴 유사도
4. **FDR (False Discovery Rate)** - 다중 검정 보정 (9,900 pairs)

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 프로젝트 루트 경로 추가
sys.path.insert(0, str(Path.cwd().parent))

from config import Config
from src.comovement import (
    calculate_ccf_matrix,
    calculate_granger_matrix,
    calculate_dtw_matrix,
    apply_fdr_to_results,
    detect_comovement_comprehensive
)

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 한글 폰트
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✓ 라이브러리 import 완료")

## 1. 데이터 로드 및 전처리

In [ ]:
# 실제 데이터 로드
df_raw = pd.read_csv(Config.DATA_RAW / 'train.csv')

print("=== 원본 데이터 정보 ===")
print(f"Shape: {df_raw.shape}")
print(f"Columns: {df_raw.columns.tolist()}")
print(f"\n고유 item_id: {df_raw['item_id'].nunique()}개")
print(f"기간: {df_raw['year'].min()}.{df_raw['month'].min()} ~ {df_raw['year'].max()}.{df_raw['month'].max()}")
print(f"\n첫 5개 행:")
display(df_raw.head())

In [ ]:
# 날짜 컬럼 생성 및 월별 집계
df_raw['date'] = pd.to_datetime(df_raw[['year', 'month']].assign(day=1))

# 같은 item_id + date에 여러 거래(seq)가 있으므로 월별로 합산
df_agg = df_raw.groupby(['date', 'item_id']).agg({
    'value': 'sum',  # 월별 총 거래액
    'weight': 'sum',  # 월별 총 중량
    'quantity': 'sum'  # 월별 총 수량
}).reset_index()

print("=== 월별 집계 후 ===")
print(f"Shape: {df_agg.shape}")
print(f"날짜 범위: {df_agg['date'].min()} ~ {df_agg['date'].max()}")
print(f"총 기간: {df_agg['date'].nunique()}개월")
print(f"\n품목별 데이터 수:")
print(df_agg.groupby('item_id').size().describe())

display(df_agg.head(10))

In [ ]:
# Long format을 Wide format으로 변환 (날짜 x 품목)
df_wide = df_agg.pivot(index='date', columns='item_id', values='value')

print("=== Wide format 변환 ===")
print(f"Shape: {df_wide.shape} (날짜 x 품목)")
print(f"\n결측값:")
missing_pct = df_wide.isnull().sum() / len(df_wide) * 100
print(f"결측값 있는 품목: {(missing_pct > 0).sum()}개")
print(f"평균 결측 비율: {missing_pct.mean():.2f}%")
print(f"최대 결측 비율: {missing_pct.max():.2f}%")

display(df_wide.head())

In [ ]:
# 결측값 처리: 0으로 채우기 (거래가 없는 경우)
df_wide_filled = df_wide.fillna(0)

print(f"결측값 처리 완료: {df_wide_filled.isnull().sum().sum()}개 결측값")
print(f"\n기본 통계:")
display(df_wide_filled.describe())

## 2. CCF (Cross-Correlation Function) 분석

품목 간 시차별 상관관계를 분석합니다.

In [ ]:
# CCF 계산 (최대 6개월 시차)
print("CCF 분석 시작...")
print(f"총 쌍 개수: {100 * 99 // 2} = 4,950 pairs")
print(f"최대 시차: {Config.CCF_LAG_MAX}개월")
print(f"임계값: {Config.CCF_THRESHOLD}")

ccf_results = calculate_ccf_matrix(
    df_wide_filled,
    max_lag=Config.CCF_LAG_MAX,
    threshold=Config.CCF_THRESHOLD
)

print(f"\nCCF 결과:")
display(ccf_results.head(20))

In [ ]:
# CCF 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# CCF 값 분포
axes[0].hist(ccf_results['max_ccf'], bins=50, edgecolor='black')
axes[0].axvline(Config.CCF_THRESHOLD, color='red', linestyle='--', label=f'Threshold={Config.CCF_THRESHOLD}')
axes[0].axvline(-Config.CCF_THRESHOLD, color='red', linestyle='--')
axes[0].set_xlabel('CCF 값')
axes[0].set_ylabel('빈도')
axes[0].set_title('CCF 분포')
axes[0].legend()

# 최적 시차 분포
axes[1].hist(ccf_results['optimal_lag'], bins=Config.CCF_LAG_MAX+1, edgecolor='black')
axes[1].set_xlabel('최적 시차 (개월)')
axes[1].set_ylabel('빈도')
axes[1].set_title('최적 시차 분포')
axes[1].set_xticks(range(Config.CCF_LAG_MAX+1))

plt.tight_layout()
plt.show()

print(f"\n유의미한 공행성 쌍: {ccf_results['is_significant'].sum()}개 ({ccf_results['is_significant'].sum()/len(ccf_results)*100:.1f}%)")

In [ ]:
# 상위 20개 공행성 쌍 출력
print("=== CCF 상위 20개 쌍 ===")
top_ccf = ccf_results.nlargest(20, 'abs_ccf')
display(top_ccf[['item_x', 'item_y', 'optimal_lag', 'max_ccf', 'is_significant']])

## 3. Granger Causality Test

방향성 있는 인과관계를 검정합니다.

In [ ]:
# 계산량이 많으므로 일부 품목만 테스트 (전체 100개면 9,900 pairs)
# 상위 CCF 쌍의 품목들로 제한
top_items = set()
for _, row in top_ccf.head(30).iterrows():
    top_items.add(row['item_x'])
    top_items.add(row['item_y'])

top_items = list(top_items)[:20]  # 상위 20개 품목만
df_wide_subset = df_wide_filled[top_items]

print(f"Granger 인과관계 검정: {len(top_items)}개 품목 선택")
print(f"총 검정 횟수: {len(top_items) * (len(top_items)-1)} (양방향)")
print(f"선택된 품목: {top_items}")

In [ ]:
# Granger 인과관계 검정 실행
print("\nGranger 검정 시작...")
granger_results = calculate_granger_matrix(
    df_wide_subset,
    max_lag=Config.GRANGER_MAX_LAG,
    p_threshold=Config.GRANGER_PVAL_THRESHOLD
)

print(f"\nGranger 결과:")
display(granger_results.head(20))

In [ ]:
# FDR 보정 적용
print("FDR 보정 적용...")
granger_results_fdr = apply_fdr_to_results(
    granger_results,
    p_value_col='p_value',
    alpha=Config.FDR_ALPHA
)

print(f"\n보정 전: {granger_results['is_causal'].sum()}개 인과관계")
print(f"보정 후: {granger_results_fdr['fdr_rejected'].sum()}개 인과관계")

display(granger_results_fdr.head(20))

In [ ]:
# FDR 보정 후 유의미한 인과관계만 추출
significant_granger = granger_results_fdr[granger_results_fdr['fdr_rejected']].copy()

print(f"=== FDR 보정 후 유의미한 인과관계 ({len(significant_granger)}개) ===")
display(significant_granger[['item_cause', 'item_effect', 'best_lag', 'p_value', 'fdr_threshold']])

## 4. DTW (Dynamic Time Warping) 분석

비선형 패턴 유사도를 측정합니다.

In [ ]:
# DTW도 계산량이 많으므로 일부 품목만
print(f"DTW 분석: {len(top_items)}개 품목")
print(f"총 쌍 개수: {len(top_items) * (len(top_items)-1) // 2}")
print(f"임계값: {Config.DTW_THRESHOLD}")

dtw_results = calculate_dtw_matrix(
    df_wide_subset,
    threshold=Config.DTW_THRESHOLD
)

print(f"\nDTW 결과:")
display(dtw_results.head(20))

In [ ]:
# DTW 거리 분포 시각화
plt.figure(figsize=(10, 6))
plt.hist(dtw_results['dtw_distance'], bins=50, edgecolor='black')
plt.axvline(Config.DTW_THRESHOLD, color='red', linestyle='--', label=f'Threshold={Config.DTW_THRESHOLD}')
plt.xlabel('DTW 거리 (정규화)')
plt.ylabel('빈도')
plt.title('DTW 거리 분포')
plt.legend()
plt.show()

print(f"\n유사 패턴 쌍: {dtw_results['is_similar'].sum()}개 ({dtw_results['is_similar'].sum()/len(dtw_results)*100:.1f}%)")

In [ ]:
# 상위 10개 유사 쌍 출력
print("=== DTW 상위 10개 유사 쌍 ===")
top_dtw = dtw_results.nsmallest(10, 'dtw_distance')  # 거리가 작을수록 유사
display(top_dtw)

## 5. 종합 분석 및 결과 저장

In [ ]:
# 결과 저장
output_dir = Config.DATA_PROCESSED
output_dir.mkdir(parents=True, exist_ok=True)

ccf_results.to_csv(output_dir / 'ccf_results.csv', index=False)
granger_results_fdr.to_csv(output_dir / 'granger_results_fdr.csv', index=False)
dtw_results.to_csv(output_dir / 'dtw_results.csv', index=False)

print(f"✓ 결과 저장 완료: {output_dir}")
print(f"  - ccf_results.csv")
print(f"  - granger_results_fdr.csv")
print(f"  - dtw_results.csv")

In [ ]:
# 종합 요약
print("=" * 60)
print("공행성 탐지 종합 요약")
print("=" * 60)
print(f"\n1. CCF 분석")
print(f"   - 총 쌍 개수: {len(ccf_results)}")
print(f"   - 유의미한 쌍 (|CCF| >= {Config.CCF_THRESHOLD}): {ccf_results['is_significant'].sum()}개")
print(f"   - 비율: {ccf_results['is_significant'].sum()/len(ccf_results)*100:.1f}%")

print(f"\n2. Granger 인과관계 (상위 {len(top_items)}개 품목)")
print(f"   - 총 검정 횟수: {len(granger_results_fdr)}")
print(f"   - FDR 보정 전 유의: {granger_results['is_causal'].sum()}개")
print(f"   - FDR 보정 후 유의: {granger_results_fdr['fdr_rejected'].sum()}개")

print(f"\n3. DTW 분석 (상위 {len(top_items)}개 품목)")
print(f"   - 총 쌍 개수: {len(dtw_results)}")
print(f"   - 유사 패턴 쌍 (거리 <= {Config.DTW_THRESHOLD}): {dtw_results['is_similar'].sum()}개")
print(f"   - 비율: {dtw_results['is_similar'].sum()/len(dtw_results)*100:.1f}%")

print("\n=" * 60)
print("✓ Phase 3.2 공행성 탐지 완료!")
print("=" * 60)